### Sistemas Não Lineares: métodos iterativos I (cap 3)

---

1. [Histórico](#Histórico)

2. [Método da bisseção](#Método-da-bisseção)
    * [Exercício 1](#Exercício-1:)
    
3. [Método de Newton](#Método-de-Newton)
    * [Exercício 2](#Exercício-2:)
    * [Exercício 3](#Exercício-3:)
    
4. [Método de Newton para sistemas de equações não lineares](#Método-de-Newton-para-sistemas-de-equações-não-lineares)
    * [Exercício 4](#Exercício-4:)
    * [Exercício 5](#Exercício-5:)

5.  [Método da secante para sistemas de equações não lineares](#Método-da-secante-para-sistemas-de-equações-não-lineares)

## 14-08

#### Histórico

Os babilônios já implementavam uma versão do método de Newton, um dos métodos estudados nessa aula, em problemas do tipo: $x^2 = a$

* Como era resolvido?

$x^2 = a \rightarrow x = \frac{a}{x}$

Logo, começavam com um chute inicial $x = x_{0}$. Se $x_{0} = \frac{a}{x_{0}}$, então chegamos no resultado. 

Senão, uma das duas parcelas da igualdade é maior que a outra, e sabemos que a raiz está entre ambas. 

Logo, pegamos a média entre elas: $x_{1} = \frac{1}{2}(x_{0} + \frac{a}{x_{0}})$

#### Método da bisseção

$f: [a,b] \rightarrow \Re$ (contínua)

Queremos descobrir $x_* : f(x_*) = 0$. 

Sob as seguintes condições: $f(a)f(b) < 0$

O tamanho do intervalo decresce em progressão geométrica, mas o ero decresce linearmente (nos dígitos! que seria o equivalente a observar o erro em log).

In [1]:
def metodo_bissecao(f, a, b, threshold=0.01, i=0):
    
    """
    Calcula a raiz da função f dado o intervalo [a,b], sob as condiçãos de f contínua e f(a)f(b) < 0.
    
    :param f: função contínua
    :param a: limite inferior do intervalo
    :param b: limite superior do intervalo
    :param threshold: erro d aproximação máximo (tamanho do intervalo)
    
    :return (x, i): (raiz, número de iterações)
    """
    
    if f(a) == 0:
        return f(a)
    
    if f(b) == 0:
        return f(b)
    
    if f(a)*f(b) < 0:
        
        while (b - a) > threshold:

            x = (a+b)/2

            if f(x) == 0:
                return x, i
            
            elif f(x) < 0:
                i+=1
                a = x
                
            elif f(x) > 0:
                i+=1
                b = x
        
        return (a+b)/2, i

#### Exercício 1:

Implemente o método da bisseção e use-o para calcular uma raiz de $f(x)=x^3−x−2$

In [2]:
x_b, i_b = metodo_bissecao(lambda x: x**3-x-2, -10, 10, 0.0001)

In [3]:
x_b, i_b

(1.5214157104492188, 18)

In [4]:
x_b**3 - x_b - 2

0.00021400397033088936

#### Método de Newton

A cada iteração, calculamos $x_{k+1} = x_{k} - \frac{f(x_k)}{f'(x_k)}$

Nesse caso, as raízes da função são os **atratores**. As **bacias de atração** são as regiões formadas ao redor dos atratores nas quais o método converge para aquela raiz. Entre as bacias de atração temos as **fronteiras**, regiões que não percentem a nenhuma bacia de atração. 
* Se nosso palpite inicial estiver numa **bacia de atração**, o método vai convergir para o atrator local;
* Geralmente, se nosso palpite inicial estiver na **fronteira**, o método não converge.

*Exemplo*:

$ f(x) = x^2 + 4$

As raízes são $-2i$, cuja bacia de atração corresponde ao semiplano abaixo da reta real, e $2i$, cuja bacia de atração corresponde ao semiplano acima da reta real. Os números reais são a fronteira! Logo, se iniciarmos com um número real, não convergimos para uma raiz da função :'(

#### Exercício 2:

Implemente o método de Newton e use-o para calcular uma raiz de $f(x)=x^3−x−2$

In [5]:
from scipy.misc import derivative

def metodo_newton(f, x, threshold=0.01, i=0):
    
    """
    Calcula a raiz da função f dado um valor inicial x.
    
    :param f: função contínua
    :param x: valor inicial
    :param threshold: erro de aproximação máximo
    
    :return (x, i): (raiz, número de iterações)
    """
    
    if f(x) == 0:
        
        return x
    
    while abs(f(x)) > threshold:
        
        i+=1
        d_x = derivative(f, x)
        x = x - f(x)/d_x
    
    return x, i

In [6]:
x_n, i_n = metodo_newton(lambda x: x**3-x-2, -10, 0.0001)

In [7]:
x_n, i_n

(1.5213904935469849, 26)

In [8]:
x_n**3 - x_n - 2

6.411464806443945e-05

#### Exercício 3:

Compare a velocidade de convergência dos métodos

**R:** Como vimos nas implementações acima, para um mesmo erro de aproximação ($\delta < 1 \times 10^{-4}$), o método da bisseção chegou à solução com erro de $\approx 2.15 \times 10^{-4}$ em 18 iterações, enquanto o método de Newton teve um erro de $\approx 6.41 \times 10^{-5}$ em 26 iterações. 

Embora o método de Newton tenha apresentado uma solução mais próxima da raiz da equação, o método da bisseção chegou a uma solução mais rápido.

## 16-08

#### [Método de Newton para sistemas de equações não lineares](http://www.math.ohiou.edu/courses/math3600/lecture13.pdf)

Temos $F : \Re^n \rightarrow \Re^n$, um sistema de equações $f_1, f_2, ..., f_n$ representado pelas linhas da matriz $F$.

Logo, sendo $r$ o vetor raiz das equações:

$F(v) = [ f_{1}(x), f_{2}(x), ..., f_{n}(x)] \rightarrow F(r) = \vec{0}$

Sendo $J_{F}$ o Jacobiano da matriz $F$ e $e_{k}$ como o erro na iteração $k$, temos que:

$F(v_{k} - e_{k}) \approx F(v_{k}) - J_{F}(v_{n})e_{k} \approx 0$, quando $v_{k} \rightarrow r$

Logo, a cada iteração atualizamos o valor de $v_k$:

$v_{k+1} = v_{k} - J_{F}(v_{k})^{-1}F(v_{k})$

**Exercício 4:**  

Use o método de Newton para determinar uma solução para

$ x + y + z = 3 \\
x^2 + y^2 + z^2 = 5 \\
e^x + xy - xz = 1$

In [9]:
import sympy as sp

def nonlin_newton(F, var, v, threshold=0.01, i=0):
    
    """
    Calcula o vetor raiz do conjunto de equações não lineares F dado um valor inicial v.
    
    :param F: matriz com as equações
    :param var: variáveis das equações
    :param v: vetor inicial
    :param threshold: erro de aproximação máximo
    
    :return (v, i): (vetor raiz, número de iterações)
    """
    
    n = F.shape[0]
    
    values = {k:v for k,v in zip(var,v)}
    
    if F.subs(values) ==  sp.Matrix([0 for i in range(n)]):
        
        return v
    
    # Calcula o Jacobiano (simbólico)
    J_F = F.jacobian(var)
    
    # Iniciamos o módulo do erro com um valor acima do threshold
    e_norm = threshold + 1 
    
    # Busca a melhor aproximação
    while e_norm > threshold:
        
        i+=1
        
        values = {k:v for k,v in zip(var,v)}
        F_v = F.subs(values)
        J_v = J_F.subs(values)
        
        last = v
        v = v - J_v.inv()*F_v
        
        e_norm = (v - last).norm()
        
    return v, i

In [10]:
import math
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')

F = sp.Matrix([x + y + z -3, x**2 + y**2 + z**2 -5, math.e**x + x*y - x*z -1])
var = sp.Matrix([x,y,z])
v = sp.Matrix([1,2,3])

In [11]:
v_sol, n = nonlin_newton(F, var, v, 0.0001)

In [12]:
v_sol, n

(Matrix([
 [-2.50251394762964e-11],
 [     2.00000000003308],
 [    0.999999999991949]]), 7)

In [13]:
values = {k:v for k,v in zip(var,v_sol)}
F.subs(values)

Matrix([
[-1.11022302462516e-16],
[ 1.16203713318441e-10],
[-5.00502325861914e-11]])

#### Exercício 5:
Mostre que o método de Newton converge quadraticamente caso o ponto inicial esteja suficientemente próximo da solução de $f(x^*) = 0$ e $f'(x^*) \neq 0$ (sob a hipótese de $f$ ser *suave*).

**R:** Seja $r$ a raiz de $f$, $x_k$ a aproximação da raiz e $e_{k} = x_{k} - r$ o erro na iteração $k$.

Logo, expandindo $f(x_{k} - e_{k})$ por série de Taylor no ponto $x_{k}$, temos:

$f(x_{k} - e_{k}) = f(x_{k}) - f'(x_{k})e_{k} + \frac{f''(\theta)e_{k}^2}{2}$, na qual o útlimo termo é o [termo complementar de Lagrange](https://pt.wikipedia.org/wiki/Teorema_de_Taylor), e $\theta \in [x_{k}, x_{k}-e_{k}]$

Como $r = x_{k} - e_{k}$, para que o método convirja temos que:

$f(x_{k} - e_{k}) = f(r) = 0 \rightarrow f(x_{k}) - f'(x_{k})e_{k} + \frac{f''(\theta)e_{k}^2}{2} = 0$

Dividindo todos os termos por $f'(x_{k})$ e rearranjando a equação, temos:

$\frac{f(x_{k})}{f'(x_{k})} - e_{k} + \frac{f''(\theta)e_{k}^2}{2f'(x_{k})} = 0 \rightarrow e_{k} = (\frac{f''(\theta)e_{k}^2}{2f'(x_{k})} + \frac{f(x_{k})}{f'(x_{k})})$

Pelo método de Newton, sabemos que:

$x_{k+1} = x_{k} - \frac{f(x_k)}{f'(x_k)} \rightarrow e_{k+1} - e_{k} = x_{k+1} - x_{k} = - \frac{f(x_k)}{f'(x_k)}$

Logo, substituindo $e_{k+1} - e_{k}$ na equação, temos:

$e_{k} = \frac{f''(\theta)e_{k})^2}{2f'(x_{k})} - (e_{k+1} - e_{k}) \rightarrow e_{k+1} = \frac{f''(\theta)e_{k})^2}{2f'(x_{k})}$

Tomando o valor absoluto em ambos os lados da equação, temos:

$ \lVert e_{k+1}\rVert = \left\lVert \frac{f''(\theta)e_{k}^2}{2f'(x_{k})} \right\rVert = \left\lVert \frac{f''(\theta)}{2f'(x_{k})} \right\rVert e_{k}^2$

Se tomarmos $x_n$ tal que $\frac{1}{2}\left|\frac {f''(x_n)}{f'(x_n)}\right| \leq C\left|{\frac {f''(r)}{f'(r)}}\right|$, para $C < \infty$, ou seja, tomando $x$ *suficiente próximo de* $r$, garantimos que $\left|{e_{n+1}}\right|\leq \left\lVert \frac{f''(\theta)}{2f'(x_{k})} \right\rVert{e_{n}}^{2}$. 

\* Consulta ao site: https://en.wikipedia.org/wiki/Newton%27s_method

#### [Método da secante para sistemas de equações não lineares](http://mathforcollege.com/nm/mws/gen/03nle/mws_gen_nle_txt_secant.pdf)